## 파이토치의 nn.Embedding()
- 임베딩 벡터를 사용하는 두가지 방법
- 1) 처음부터 임베딩 벡터를 학습하는 방법
- 2) 미리 사전에 훈련된 임베딩 벡터를 가져와 사용하는 방법

### 1. 임베딩 층은 룩업 테이블
- 1) 입력 시퀀스의 각 단어들은 모두 정수 인코딩
- 2) 어떤 단어 -> 단어에 부여된 고유한 정수값 -> 임베딩 층 통과 -> 밀집벡터
- 3) 입력 정수에 대해 밀집벡터로 맵핑
- 4) 밀집벡터를 학습 --> 밀집벡터를 임베딩벡터라함<br>
<br>
- 특정 단어와 맵핑되는 정수를 입덱스로 가지는 테이블로부터 임베딩 벡터값을 가져오는 룩업테이블
- 룩업테이블은 단어 집합의 크기만큼의 행을 가지므로 모든 단어는 고유한 임베딩 벡터를 가짐
![image.png](https://github.com/tenjumh/PyTorch/blob/master/image/lookup_table.png?raw=True)

### 룩업 테이블 과정을 nn.Embedding()을 사용하지 않고 구현

In [20]:
import torch

In [21]:
train_data = 'you need to know how to code'
word_set = list(train_data.split())  
word_set = sorted(set(word_set))
print(word_set)
print()
vocab = {}
vocab['<UNK>'] = 0
vocab['<PAD>'] = 1
for idx, word in enumerate(word_set):
    vocab[word] = idx + 2
    
print(vocab)

['code', 'how', 'know', 'need', 'to', 'you']

{'<UNK>': 0, '<PAD>': 1, 'code': 2, 'how': 3, 'know': 4, 'need': 5, 'to': 6, 'you': 7}


OR
```
train_data = 'you need to know how to code'
word_set = set(train_data.split()) 
vocab = {word: i+2 for i, word in enumerate(word_set)}  # 단어 집합의 각 단어에 고유한 정수 맵핑.
vocab['<unk>'] = 0
vocab['<pad>'] = 1
print(vocab)
```

In [22]:
# 단어 집합의 크기만큼의 행을 가지는 테이블 생성.
embedding_table = torch.FloatTensor([
                               [ 0.0,  0.0,  0.0],
                               [ 0.0,  0.0,  0.0],
                               [ 0.2,  0.9,  0.3],
                               [ 0.1,  0.5,  0.7],
                               [ 0.2,  0.1,  0.8],
                               [ 0.4,  0.1,  0.1],
                               [ 0.1,  0.8,  0.9],
                               [ 0.6,  0.1,  0.1]])

In [27]:
# 임의의 문장 "you need to run"에 대해서 룩업 테이블을 통해 임베딩 벡터들 가져옴
sample = 'you need to run'.split()
idxes = []
for word in sample:
    try:
        idxes.append(vocab[word])
    except KeyError:   # vocab에 없는 단어의 경우 KeyError가 발생하니 <UNK> 예외처리
        idxes.append(vocab['<UNK>'])
idxes = torch.LongTensor(idxes)
print(idxes)
# 룩업 테이블
lookup_result = embedding_table[idxes, :]   # 각 정수를 인덱스로 임베딩 테이블에서 값을 가져옴
print(lookup_result)

tensor([7, 5, 6, 0])
tensor([[0.6000, 0.1000, 0.1000],
        [0.4000, 0.1000, 0.1000],
        [0.1000, 0.8000, 0.9000],
        [0.0000, 0.0000, 0.0000]])


### 룩업 테이블 과정을 nn.Embedding()을 사용 구현
- num_embeddings : 임베딩을 할 단어들의 개수, 단어 집합의 크기
- embedding_dim : 임베딩 할 벡터의 차원, 하이퍼파라미터
- padding_idx : 선택적 사용 인자, 패딩을 위한 토큰의 인덱스 알려줌

In [32]:
import torch
import torch.nn as nn

In [33]:
train_data = 'you need to know how to code'
word_set = list(train_data.split())  
word_set = sorted(set(word_set))
vocab = {}
vocab['<UNK>'] = 0
vocab['<PAD>'] = 1
for idx, word in enumerate(word_set):
    vocab[word] = idx + 2
print(len(vocab))

8


In [34]:
embedding_layer = nn.Embedding(num_embeddings=len(vocab),
                              embedding_dim=3,
                              padding_idx = 1)

In [35]:
print(embedding_layer)

Embedding(8, 3, padding_idx=1)


In [36]:
print(embedding_layer.weight)

Parameter containing:
tensor([[-1.8918, -0.5395, -0.7265],
        [ 0.0000,  0.0000,  0.0000],
        [-1.0697, -0.0470, -1.3543],
        [-0.8331,  1.0396, -0.1173],
        [ 0.1322, -1.7347,  1.0463],
        [ 0.6788,  0.8714,  1.3568],
        [ 1.1479, -0.7493,  1.7126],
        [-0.2333,  2.1249, -0.1547]], requires_grad=True)
